In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Nomenclature
- Complete data collection procedure, from init to writing is referred to as a "run".   
- Each step in a run is an "event".  
- Where a run is in the sequence of events is it's "state"
- To move between states requires "triggers"

## States are
* waiting: in between events; triggered by conclusion of steps.  Options while waiting include
    * set parameters (wavelength/steps, integration time, devices to include in the initialization, compression?, write results?, display in real time?)
    * initialize
    * run all
* initializing (complete and between runs): triggered by initialize command (at any time, but warning if initialize already happened) will initialize all devices with checked boxes in GUI. Returns dictionary of device metadata
* collecting: triggered by collection command (at end sets self.data_in_queue = True)
* compressing: triggered by compression commmand (at end sets self.data_ready_to_store = True) 
* writing: triggered by writing command
* fubar: triggered by an error in any of the steps
* interupt: is this a state or a command?  

In [2]:
import os
import sys
import pdb
import numpy as np
import copy
from math import *

sys.path.append(r'..\pylab\pylabsm')
sys.path.append(r'..\pylab\pylabcal\pylabcald')

import sys
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

#import pylab
from pylabsm import SM
from pylabsm import SpectralCalibrationMachine
from pylabcaldlib.utils import parameters
from pylabcaldlib.instruments.repeatedtimer import RepeatedTimer
from pylabcaldlib.instruments.powermaxusb import PowermaxUSB
from pylabcaldlib.instruments.serial_motor_dpy50601 import DPY50601

# GUI Developement

In [1]:
import os
import sys
import pdb
import numpy as np
import copy
from math import *
import datetime as datetime

sys.path.append(r'..\pylab\pylabsm')
sys.path.append(r'..\pylab\pylabcal\pylabcald')

import sys
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

#import pylab
from pylabsm import SM
from pylabsm import SpectralCalibrationMachine
from pylabcaldlib.utils import parameters
from pylabcaldlib.instruments.repeatedtimer import RepeatedTimer
from pylabcaldlib.instruments.powermaxusb import PowermaxUSB
from pylabcaldlib.instruments.serial_motor_dpy50601 import DPY50601

class tabdemo(QTabWidget):
    def __init__(self, parent = None):
        super(tabdemo, self).__init__(parent)
        
        self.state_machine = SpectralCalibrationMachine()
        self.parameters_first = self.state_machine.params.copy()
        self.parameters_last = self.state_machine.params.copy()
        self.parameters = self.state_machine.params.copy()
        
        self.tab1 = QWidget()
        self.tab2 = QWidget()
        self.tab3 = QWidget()

        self.addTab(self.tab1,"Run")
        self.addTab(self.tab2,"Settings")
        self.addTab(self.tab3,"Metadata")
        self.tab1UI()
        self.tab2UI()
        self.tab3UI()
        self.setWindowTitle("Spectral Calibration Interface")
        
        self.retranslateUi()
        
    def tab1UI(self):
        layout = QGridLayout()
        self.setTabText(0,"Run")
        
        # Initialize Button
        self.initialize_button = QPushButton(self)
        self.initialize_button.setText("Initialize")
        layout.addWidget(self.initialize_button, 0, 0)

        # Start Run Button
        self.start_run_button = QPushButton(self)
        self.start_run_button.setText("Start Run")
        layout.addWidget(self.start_run_button, 1, 0)
        
        # Automate Check Box (default is checked)
        self.automate_box = QCheckBox(self)
        #self.automate_box.setText("Automate")
        self.automate_box.setText("Use .cfg")
        layout.addWidget(self.automate_box, 1, 1)
        self.automate_box.setChecked(True)
        
        # Event progress bar
        self.progress = QProgressBar(self)
        layout.addWidget(self.progress, 2, 0, 1, 2)
        
        # Full run progress bar
        self.progress = QProgressBar(self)
        layout.addWidget(self.progress, 3, 0, 1, 2)
        
        # Message box
        self.msg = QTextBrowser()
        layout.addWidget(self.msg, 0, 2, 4, 3)
        
        # Populate Tab 1
        self.tab1.setLayout(layout)
        
    def tab2UI(self):
        layout = QFormLayout()
        self.setTabText(1,"Settings")       

        # Monochrometer Wavelengths
        layout2 = QGridLayout()
        label = QLabel("Wavelengths: ")
        label_0 = QLabel("Monochrometer")
        label_l = QLabel("Start ")
        label_m = QLabel("End ")
        label_r = QLabel("Step ")
        layout2.addWidget(label_0, 1, 0)
        layout2.addWidget(label_l, 1, 1)
        layout2.addWidget(label_m, 1, 2)
        layout2.addWidget(label_r, 1, 3)
        
        wavelength_start = int(self.state_machine.params['monochrometer']['wavelength_start'])
        wavelength_stop = int(self.state_machine.params['monochrometer']['wavelength_stop'])
        wavelength_step = int(self.state_machine.params['monochrometer']['wavelength_step'])
        
        self.wavelength0 = QSpinBox(self)
        self.wavelength0.setMinimum(wavelength_start)
        self.wavelength0.setMaximum(wavelength_stop)
        self.wavelength0.setValue(wavelength_start)
        self.wavelength0.valueChanged.connect(self.updateStartWavelength)
        
        self.wavelength1 = QSpinBox(self)
        self.wavelength1.setMinimum(wavelength_start)
        self.wavelength1.setMaximum(wavelength_stop)
        self.wavelength1.setValue(wavelength_stop) 
        self.wavelength1.valueChanged.connect(self.updateStopWavelength)
        
        self.wavelength2 = QSpinBox(self)
        self.wavelength2.setMinimum(1)
        self.wavelength2.setMaximum(200)
        self.wavelength2.setValue(wavelength_step) 
        self.wavelength2.valueChanged.connect(self.updateStepWavelength)

        layout2.addWidget(label, 2, 0)
        layout2.addWidget(self.wavelength0, 2, 1)
        layout2.addWidget(self.wavelength1, 2, 2)
        layout2.addWidget(self.wavelength2, 2, 3)

        # Add wavelengths Row
        layout.addRow(layout2)
        
        # Temporary Storage Directory
        layout_temp_dir = QHBoxLayout()
        label_temp_dir = QLabel("Temporary Storage Path: ")
        layout_temp_dir.addWidget(label_temp_dir)
        cfg_temp_dir = self.state_machine.params['io']['temp_storage_folder']
        self.temp_dir = QLineEdit(cfg_temp_dir)
        self.temp_dir.selectAll()
        layout_temp_dir.addWidget(self.temp_dir)   
        layout.addRow(layout_temp_dir)
        
        # Final Storage Path
        layout_final_dir = QHBoxLayout()
        label_final_dir = QLabel("Temporary Storage Path: ")
        layout_final_dir.addWidget(label_final_dir)
        cfg_final_dir = self.state_machine.params['io']['permanent_storage_folder']
        self.final_dir = QLineEdit(cfg_final_dir)
        self.final_dir.selectAll()
        layout_final_dir.addWidget(self.final_dir)   
        layout.addRow(layout_final_dir)
        
        # Suffix Name
        layout_suffix = QHBoxLayout()
        label_suffix = QLabel("Filename Suffix: ")
        layout_suffix.addWidget(label_suffix)
        cfg_suffix = self.state_machine.params['io']['suffix']
        if cfg_suffix == "date":
            cfg_suffix = self.get_time_now()
        self.suffix = QLineEdit(cfg_suffix)
        self.suffix.selectAll()
        layout_suffix.addWidget(self.suffix)   
        layout.addRow(layout_suffix)
        
        # Data Format Type (orig)
        layout3 = QGridLayout()
        label_write = QLabel("Data Format: ")
        self.csv_box = QCheckBox(self)
        self.csv_box.setText(".csv")
        self.json_box = QCheckBox(self)
        self.json_box.setText(".json")
        self.parquet_box = QCheckBox(self)
        self.parquet_box.setText(".parquet")
        layout3.addWidget(label_write, 0, 0)
        layout3.addWidget(self.csv_box, 0, 1)
        layout3.addWidget(self.json_box, 0, 2)
        layout3.addWidget(self.parquet_box, 0, 3)
        self.parquet_box.setChecked(self.pick_format('io','storage_type',".parquet"))
        self.csv_box.setChecked(self.pick_format('io','storage_type',".csv"))
        self.json_box.setChecked(self.pick_format('io','storage_type',".json"))
        # calling the uncheck method if any check box state is changed 
        self.csv_box.stateChanged.connect(self.uncheck_write) 
        self.json_box.stateChanged.connect(self.uncheck_write) 
        self.parquet_box.stateChanged.connect(self.uncheck_write) 
        layout.addRow(layout3)
        
        # Compression Type (original)
        layout4 = QGridLayout()
        label_comp = QLabel("Compression: ")
        self.zip_box = QCheckBox(self)
        self.zip_box.setText(".zip")
        self.tgz_box = QCheckBox(self)
        self.tgz_box.setText(".tgz")
        self.tar_box = QCheckBox(self)
        self.tar_box.setText(".tar")
        layout4.addWidget(label_comp, 0, 0)
        layout4.addWidget(self.zip_box, 0, 1)
        layout4.addWidget(self.tgz_box, 0, 2)
        layout4.addWidget(self.tar_box, 0, 3)
        self.zip_box.setChecked(self.pick_format('io','compression_type',".zip"))
        self.tgz_box.setChecked(self.pick_format('io','compression_type',".tgz"))
        self.tar_box.setChecked(self.pick_format('io','compression_type',".tar"))
        # calling the uncheck method if any check box state is changed 
        self.zip_box.stateChanged.connect(self.uncheck_comp) 
        self.tar_box.stateChanged.connect(self.uncheck_comp) 
        self.tgz_box.stateChanged.connect(self.uncheck_comp) 
        layout.addRow(layout4)
        
        # Populate Tab 2
        self.tab2.setLayout(layout)
        
        # DELETE
        ## Data Fromat Type (radio)
        #store = QHBoxLayout()
        #self.par_box = QRadioButton(".parquet")
        #self.csv_box = QRadioButton(".csv")
        #self.json_box = QRadioButton(".json")
        #self.par_box.setChecked(self.pick_format('storage_type',".parquet"))
        #self.csv_box.setChecked(self.pick_format('storage_type',".csv"))
        #self.json_box.setChecked(self.pick_format('storage_type',".json"))
        #store.addWidget(self.par_box)
        #store.addWidget(self.csv_box)
        #store.addWidget(self.json_box)
        #layout.addRow("Data Format: ", store)
        
        ## Compression Type (radio)
        #comp = QHBoxLayout()
        #zip_box = QRadioButton(".zip")
        #tgz_box = QRadioButton(".tgz")
        #tar_box = QRadioButton(".tar")
        #zip_box.setChecked(self.pick_format('storage_type',".zip"))
        #tgz_box.setChecked(self.pick_format('storage_type',".tgz"))
        #tar_box.setChecked(self.pick_format('storage_type',".tar"))
        #comp.addWidget(zip_box)
        #comp.addWidget(tgz_box)
        #comp.addWidget(tar_box)
        #tgz_box.setChecked(True)
        #layout.addRow("Compression: ", comp)

        # Populate Tab 2
        #self.tab2.setLayout(layout)
        
    def tab3UI(self):
        layout = QVBoxLayout()
        self.setTabText(2,"Metadata")
        
        # Metadata Checkboxes
        layout.addWidget(QLabel("Metadata to Include")) 
        self.cryo_temperature = QCheckBox("Cryo_Temperature")
        layout.addWidget(self.cryo_temperature)
        self.cryo_temperature.setChecked(self.pick_format('metadata','cryo_temperature',"True"))
        self.power_meter = QCheckBox("Power_Meter")
        layout.addWidget(self.power_meter)
        self.power_meter.setChecked(self.pick_format('metadata','power_meter',"True"))
        self.filter_wheel_1 = QCheckBox("Filter_Wheel_1")
        layout.addWidget(self.filter_wheel_1)
        self.filter_wheel_1.setChecked(self.pick_format('metadata','filter_wheel_1',"True"))
        self.filter_wheel_2 = QCheckBox("Filter_Wheel_2")
        layout.addWidget(self.filter_wheel_2)
        self.filter_wheel_2.setChecked(self.pick_format('metadata','filter_wheel_2',"True"))
        self.encoder_1 = QCheckBox("Encoder_1")
        layout.addWidget(self.encoder_1)
        self.encoder_1.setChecked(self.pick_format('metadata','encoder_1',"True"))
        self.encoder_2 = QCheckBox("Encoder_2")
        layout.addWidget(self.encoder_2)
        self.encoder_2.setChecked(self.pick_format('metadata','encoder_2',"True"))
        
        # Keywords
        label_keywords = QLabel("Search Keywords: ")
        layout.addWidget(label_keywords)
        cfg_keywords = cfg_final_dir = self.state_machine.params['io']['keywords']
        self.keywords = QLineEdit(cfg_keywords)
        #self.keywords.selectAll()
        #self.keywords.returnPressed(self.update_config_io("keywords"))
        layout.addWidget(self.keywords)        
        
        
        # Populate Tab 3
        self.tab3.setLayout(layout)
        

    def retranslateUi(self):
        
        self.initialize_button.clicked.connect(self.initialize_button_clicked)
        
        self.start_run_button.clicked.connect(self.start_run_button_clicked)

        #pdb.set_trace()
        if len(self.state_machine.message_box) > 0:
            for i in self.state_machine.message_box:
                last_line = self.state_machine.message_box.pop()
                self.msg.append(last_line)
    
    def initialize_button_clicked(self):
        
        self.state_machine = SpectralCalibrationMachine()
        self.display_messages_in_box()
    
    def update_config_io(self, values):
        pdb.set_trace()
        self.state_machine.params['io']['keywords'] = values
        self.automate_box.setChecked(False)
        
    def updateStartWavelength(self, values):
        self.state_machine.params['monochrometer']['wavelength_start'] = values
        self.automate_box.setChecked(False)
        msg_txt = 'Starting Wavelength Parameter Changed to %4i' % values
        self.state_machine.message_box.append(msg_txt)
        self.display_messages_in_box()
        
    def updateStopWavelength(self, values):
        self.state_machine.params['monochrometer']['wavelength_stop'] = values
        self.automate_box.setChecked(False)
        msg_txt = 'Ending Wavelength Parameter Changed to %4i' % values
        self.state_machine.message_box.append(msg_txt)
        self.display_messages_in_box()
        
    def updateStepWavelength(self, values):
        self.state_machine.params['monochrometer']['wavelength_step'] = values
        self.automate_box.setChecked(False)
        msg_txt = 'Step Wavelength Parameter Changed to %4i' % values
        self.state_machine.message_box.append(msg_txt)
        self.display_messages_in_box()
        
    def start_run_button_clicked(self):
        
        self.state_machine.transduce([{'initialize': [0,2,4]},{'collect': [0,2,4]},{'compress': [0,2,4]},{'write_to_disk': [0,2,4]}])
        self.display_messages_in_box()

    def progress_bar_estimator(self):
        
        self.completed = 0

        while self.completed < 100:
            self.completed += 0.0001
            self.progress.setValue(self.completed)
            
    def display_messages_in_box(self):

        while len(self.state_machine.message_box) > 0 :
            last_line = self.state_machine.message_box.pop(0)
            self.state_machine.message_log.append(last_line)
            self.msg.append(last_line)
    
    def get_time_now(self):
        dateTimeObj = datetime.datetime.now()
        #return "_" + dateTimeObj.strftime("%H:%M:%S")
        return dateTimeObj.strftime("_%d%m%Y_%H%M")

    def pick_format(self, format_level, format_type, format_in):
        return self.state_machine.params[format_level][format_type].lower() == format_in.lower()
        
    # uncheck method 
    def uncheck_comp(self, state):
        self.automate_box.setChecked(False)
  
        # checking if state is checked 
        if state == Qt.Checked: 
  
            # if first check box is selected 
            if self.sender() == self.zip_box: 
  
                # making other check box to uncheck 
                self.tgz_box.setChecked(False) 
                self.tar_box.setChecked(False) 
  
            # if second check box is selected 
            elif self.sender() == self.tar_box: 
  
                # making other check box to uncheck 
                self.zip_box.setChecked(False) 
                self.tgz_box.setChecked(False) 
  
            # if third check box is selected 
            elif self.sender() == self.tgz_box: 
  
                # making other check box to uncheck 
                self.zip_box.setChecked(False) 
                self.tar_box.setChecked(False) 
            
    # uncheck method 
    def uncheck_write(self, state):
        self.automate_box.setChecked(False)
  
        # checking if state is checked 
        if state == Qt.Checked: 
  
            # if first check box is selected 
            if self.sender() == self.csv_box: 
  
                # making other check box to uncheck 
                self.json_box.setChecked(False) 
                self.parquet_box.setChecked(False) 
  
            # if second check box is selected 
            elif self.sender() == self.json_box: 
  
                # making other check box to uncheck 
                self.csv_box.setChecked(False) 
                self.parquet_box.setChecked(False) 
  
            # if third check box is selected 
            elif self.sender() == self.parquet_box: 
  
                # making other check box to uncheck 
                self.csv_box.setChecked(False) 
                self.json_box.setChecked(False) 
            
#def main():
#    app = QApplication(sys.argv)
#    ex = tabdemo()
#    ex.show()
#    sys.exit(app.exec_())

#if __name__ == '__main__':
#    main()

In [ ]:
app = QApplication(sys.argv)
form = tabdemo()
form.show()
sys.exit(app.exec_())

Start by Initializing
Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...
Start by Initializing
Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...
Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
PowerMax USB Sensor not found...
Compressing Data
Writing Data to Disk


In [ ]:
form.state_machine.params['monochrometer']

In [ ]:
form.state_machine.params['io']

In [ ]:
form.state_machine.params['metadata']

In [ ]:
form.state_machine.message_log

In [ ]:
class MainWindow(QMainWindow):

    def __init__(self, parent=None):
        super(MainWindow, self).__init__(parent)
        
        self.state_machine = SpectralCalibrationMachine()
        #self.parameters_first = self.state_machine.params.copy()
        #self.parameters_last = self.state_machine.params.copy()
        #self.parameters = self.state_machine.params.copy()
        
        self.setWindowTitle("Spectral Calibration Interface")
        
        layout = QGridLayout()
        
        # Left Side 
        
        self.initialize_button = QPushButton(self)
        self.initialize_button.setText("Initialize")
        layout.addWidget(self.initialize_button, 0, 0)
        
        self.setup_button = QPushButton(self)
        self.setup_button.setText("Setup")
        layout.addWidget(self.setup_button, 1, 0)

        self.automate_box = QCheckBox(self)
        self.automate_box.setText("Automate")
        layout.addWidget(self.automate_box, 1, 1)
        self.automate_box.setChecked(True)

        self.start_run_button = QPushButton(self)
        self.start_run_button.setText("Start Run")
        layout.addWidget(self.start_run_button, 2, 0)
                
        label = QLabel("Wavelength: ")
        self.wavelength_select = QLabel(self.state_machine.params['monochrometer']['wavelength_start'],self)
        layout.addWidget(label, 3, 0)
        layout.addWidget(self.wavelength_select, 3, 1)
        
        # Right Side
        
        self.msg = QTextBrowser()
        layout.addWidget(self.msg, 0, 2, 4, 3)
        
        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        
        self.retranslateUi()
        
        
    def updateLabel(self, value):

        self.wavelength_select.setText(str(value))
        
    def retranslateUi(self):
        
        self.initialize_button.clicked.connect(self.initialize_button_clicked)
        
        self.setup_button.clicked.connect(self.setup_button_clicked)
        
        self.start_run_button.clicked.connect(self.start_run_button_clicked)
        #self.button2.clicked.connect(self.initialize_button_clicked)
        
        
        self.dialogTextBrowser = setupWindow(self)
        
        #pdb.set_trace()
        if len(self.state_machine.message_box) > 0:
            for i in self.state_machine.message_box:
                last_line = self.state_machine.message_box.pop()
                self.msg.append(last_line)
        
    def initialize_button_clicked(self):
        
        self.state_machine = SpectralCalibrationMachine()

        
    @pyqtSlot()
    def setup_button_clicked(self):
        
        tst = self.dialogTextBrowser.exec_()
        if tst:
            print('OK')
            self.state_machine.message_box.append('Setup Vals Changed')
            self.updateLabel(self.state_machine.params['monochrometer']['wavelength_start'])
            self.parameters_last = self.parameters
        else:
            print('Cancel')
            self.parameters = self.parameters_last
            self.state_machine.params = self.parameters_last
        #pdb.set_trace()

        
    def start_run_button_clicked(self):
        
        state_machine.transduce([{'initialize': [0,2,4]},{'collect': [0,2,4]},{'compress': [0,2,4]},{'write_to_disk': [0,2,4]}])

        
class setupWindow(QDialog):
    def __init__(self, obj_in, parent=None):
        super(setupWindow, self).__init__(parent)

        self.state_machine = obj_in.state_machine
        self.setWindowTitle("Settings")

        layout = QGridLayout()
        
        self.metadata_button = QPushButton(self)
        self.metadata_button.setText("Metadata to Include")
        layout.addWidget(self.metadata_button, 0, 0)
        self.metadata_button.clicked.connect(self.metadata_button_clicked)
        
        label = QLabel("Wavelengths: ")
        label_l = QLabel("Start ")
        label_m = QLabel("End ")
        label_r = QLabel("Step ")
        layout.addWidget(label_l, 1, 1)
        layout.addWidget(label_m, 1, 2)
        layout.addWidget(label_r, 1, 3)
        #pdb.set_trace()
        wavelength_start = int(self.state_machine.params['monochrometer']['wavelength_start'])
        wavelength_stop = int(self.state_machine.params['monochrometer']['wavelength_stop'])
        wavelength_step = int(self.state_machine.params['monochrometer']['wavelength_step'])
        self.wavelength0 = QSpinBox(self)
        self.wavelength0.setMinimum(wavelength_start)
        self.wavelength0.setMaximum(wavelength_stop)
        self.wavelength0.setValue(wavelength_start)
        self.wavelength0.valueChanged.connect(self.updateStartWavelength)
        
        self.wavelength1 = QSpinBox(self)
        self.wavelength1.setMinimum(wavelength_start)
        self.wavelength1.setMaximum(wavelength_stop)
        self.wavelength1.setValue(wavelength_stop) 
        self.wavelength1.valueChanged.connect(self.updateStopWavelength)
        
        self.wavelength2 = QSpinBox(self)
        self.wavelength2.setMinimum(wavelength_start)
        self.wavelength2.setMaximum(wavelength_stop)
        self.wavelength2.setValue(wavelength_step) 
        self.wavelength2.valueChanged.connect(self.updateStepWavelength)

        layout.addWidget(label, 2, 0)
        layout.addWidget(self.wavelength0, 2, 1)
        layout.addWidget(self.wavelength1, 2, 2)
        layout.addWidget(self.wavelength2, 2, 3)
        
        label_write = QLabel("Data Format: ")
        self.csv_box = QCheckBox(self)
        self.csv_box.setText(".csv")
        self.json_box = QCheckBox(self)
        self.json_box.setText(".json")
        self.parquet_box = QCheckBox(self)
        self.parquet_box.setText(".parquet")
        layout.addWidget(label_write, 3, 0)
        layout.addWidget(self.csv_box, 3, 1)
        layout.addWidget(self.json_box, 3, 2)
        layout.addWidget(self.parquet_box, 3, 3)
        self.parquet_box.setChecked(True)
        
        # calling the uncheck method if any check box state is changed 
        self.csv_box.stateChanged.connect(self.uncheck_write) 
        self.json_box.stateChanged.connect(self.uncheck_write) 
        self.parquet_box.stateChanged.connect(self.uncheck_write) 
        
        label_comp = QLabel("Compression: ")
        self.zip_box = QCheckBox(self)
        self.zip_box.setText(".zip")
        self.tgz_box = QCheckBox(self)
        self.tgz_box.setText(".tgz")
        self.tar_box = QCheckBox(self)
        self.tar_box.setText(".tar")
        layout.addWidget(label_comp, 4, 0)
        layout.addWidget(self.zip_box, 4, 1)
        layout.addWidget(self.tgz_box, 4, 2)
        layout.addWidget(self.tar_box, 4, 3)
        self.tgz_box.setChecked(True)
        
        # calling the uncheck method if any check box state is changed 
        self.zip_box.stateChanged.connect(self.uncheck_comp) 
        self.tar_box.stateChanged.connect(self.uncheck_comp) 
        self.tgz_box.stateChanged.connect(self.uncheck_comp) 
        
        label_suffix = QLabel("Custom Suffix: ")
        layout.addWidget(label_suffix, 5, 0)
        self.suffix = QLineEdit("_default")
        self.suffix.selectAll()
        layout.addWidget(self.suffix, 5, 1, 1, 3)
        
        self.widget = QWidget()
        self.widget.setLayout(layout)
        #self.setCentralWidget(widget)
        
        QBtn = QDialogButtonBox.Ok | QDialogButtonBox.Apply | QDialogButtonBox.Cancel
        self.buttonBox = QDialogButtonBox(QBtn)
        #self.buttonBox = QDialogButtonBox(self)
        self.buttonBox.setOrientation(Qt.Horizontal)
        #self.buttonBox.setStandardButtons(QDialogButtonBox.Cancel|QDialogButtonBox.Apply|QDialogButtonBox.Ok)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)
        
        self.verticalLayout = QVBoxLayout(self)
        self.verticalLayout.addWidget(self.metadata_button)
        self.verticalLayout.addWidget(self.widget)
        self.verticalLayout.addWidget(self.buttonBox)        
        
        # Open Metadata Window
        self.dialogTextBrowser = metadataWindow(self)

    @pyqtSlot()
    def metadata_button_clicked(self):
        self.dialogTextBrowser.exec_()
        
    def updateStartWavelength(self, values):
        self.state_machine.params['monochrometer']['wavelength_start'] = values
        #self.message_box.append('Starting Wavelength Parameter Updated')
        
    def updateStopWavelength(self, values):
        self.state_machine.params['monochrometer']['wavelength_stop'] = values
        
    def updateStepWavelength(self, values):
        self.state_machine.params['monochrometer']['wavelength_step'] = values
    
    def button_clicked(self, s):
        print("click", s)

        dlg = CustomDialog()  # If you pass self, the dialog will be centered over the main window as before.
        if dlg.exec_():
            print("Success!")
        else:
            print("Cancel!")     
    
    # uncheck method 
    def uncheck_comp(self, state): 
  
        # checking if state is checked 
        if state == Qt.Checked: 
  
            # if first check box is selected 
            if self.sender() == self.zip_box: 
  
                # making other check box to uncheck 
                self.tgz_box.setChecked(False) 
                self.tar_box.setChecked(False) 
  
            # if second check box is selected 
            elif self.sender() == self.tar_box: 
  
                # making other check box to uncheck 
                self.zip_box.setChecked(False) 
                self.tgz_box.setChecked(False) 
  
            # if third check box is selected 
            elif self.sender() == self.tgz_box: 
  
                # making other check box to uncheck 
                self.zip_box.setChecked(False) 
                self.tar_box.setChecked(False) 
            
    # uncheck method 
    def uncheck_write(self, state): 
  
        # checking if state is checked 
        if state == Qt.Checked: 
  
            # if first check box is selected 
            if self.sender() == self.csv_box: 
  
                # making other check box to uncheck 
                self.json_box.setChecked(False) 
                self.parquet_box.setChecked(False) 
  
            # if second check box is selected 
            elif self.sender() == self.json_box: 
  
                # making other check box to uncheck 
                self.csv_box.setChecked(False) 
                self.parquet_box.setChecked(False) 
  
            # if third check box is selected 
            elif self.sender() == self.parquet_box: 
  
                # making other check box to uncheck 
                self.csv_box.setChecked(False) 
                self.json_box.setChecked(False) 
            
class metadataWindow(QDialog):
    def __init__(self, parent=None):
        super(metadataWindow, self).__init__(parent)

        self.setWindowTitle("Metadata to Include")
        layout = QGridLayout()

        self.temperature_box = QCheckBox(self)
        self.temperature_box.setText("Cryo_Temperature")
        self.temperature_box.setChecked(True) 
        layout.addWidget(self.temperature_box, 1, 0)

        self.power_meter_box = QCheckBox(self)
        self.power_meter_box.setText("Power_Meter")
        self.power_meter_box.setChecked(True) 
        layout.addWidget(self.power_meter_box, 2, 0)

        self.filter_box = QCheckBox(self)
        self.filter_box.setText("Filters")
        self.filter_box.setChecked(True) 
        layout.addWidget(self.filter_box, 3, 0)
        
        # Keywords
        label_keywords = QLabel("Search Keywords: ")
        layout.addWidget(label_keywords, 0, 0)
        self.suffix = QLineEdit("Test, Development")
        self.suffix.selectAll()
        layout.addWidget(self.suffix, 0, 1, 1, 3)
        
        self.widget = QWidget()
        self.widget.setLayout(layout)
        
        self.buttonBox = QDialogButtonBox(self)
        self.buttonBox.setOrientation(Qt.Horizontal)
        self.buttonBox.setStandardButtons(QDialogButtonBox.Cancel|QDialogButtonBox.Apply|QDialogButtonBox.Ok)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)
        
        self.verticalLayout = QVBoxLayout(self)
        self.verticalLayout.addWidget(self.widget)
        self.verticalLayout.addWidget(self.buttonBox)   
        
            
    def updateMetadata(self, values):
        
        pdb.set_trace()
        self.state_machine.params['monochrometer'] = values

In [ ]:
app = QApplication(sys.argv)
form = MainWindow()
form.state_machine.params['monochrometer']
form.show()
app.exec_()

In [ ]:
form.state_machine.params['monochrometer']